In [48]:
import pandas as pd 
import numpy as np 
import scipy as sp 
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression

In [49]:
file = "data/train_tweets.txt"
temp = []
with open(file, 'r') as data:
    for line in data:
        row = []
        line = line.replace('\t'," ")
        elem = line.strip().split(" ")
        row.append(elem[0])
        row.append(" ".join(elem[1:]))
        temp.append(row) 

In [50]:
tw = pd.DataFrame(temp,columns = ["User","Tweet"])
temp = []
elem = []

In [51]:
tw.head()

,User,Tweet
0,8746,@handle Let's try and catch up live next week!
1,8746,Going to watch Grey's on the big screen - Thur...
2,8746,@handle My pleasure Patrick....hope you are well!
3,8746,@handle Hi there! Been traveling a lot and lot...
4,8746,RT @handle Looking to Drink Clean & Go Green? ...


In [52]:
tw.tail()

,User,Tweet
328927,4357,Steelbox Demonstrates Open Video Framework wit...
328928,4357,Small Businesses Rely on Sage to Help Them Rid...
328929,4357,TimeSight Systems™ Announces Next-Generation P...
328930,4357,Diebold Makes Its Leading Monitoring Solutions...
328931,4357,GVI Security Solutions to Introduce AutoIP™ VM...


In [53]:
tw.shape

(328932, 2)

In [54]:
X = tw.Tweet
y = tw.User
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [81]:
lr = LogisticRegression()
nb = MultinomialNB()
def test_model(vect, model):
    X_train_dtm = vect.fit_transform(X_train)
    print('Features: ', X_train_dtm.shape[1])
    X_test_dtm = vect.transform(X_test)
    if model == 'LR':
        lr.fit(X_train_dtm, y_train)
        y_pred_class = lr.predict(X_test_dtm)
        algorithm = 'Logistic Regression'
    elif model == 'MNB':
        nb.fit(X_train_dtm, y_train)
        y_pred_class = nb.predict(X_test_dtm)
        algorithm = 'Multinomial Naive Bayes'
    print('Accuracy: ', metrics.accuracy_score(y_test, y_pred_class))
    print(algorithm)

In [ ]:
vectorizer = CountVectorizer(stop_words='english')
test_model(vectorizer, model='MNB')
test_model(vectorizer, model='LR')

Features:  190412
Accuracy:  0.12560650833606946
Multinomial Naive Bayes
Features:  190412


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
